In [21]:
# To download the html of a page that shows the standings of the EPL
import requests

In [22]:
# Define the url that will be scrapped
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [23]:
# Download the page
data = requests.get(standings_url)

In [24]:
# To parse html
from bs4 import BeautifulSoup

In [25]:
# Initilise soup object
soup = BeautifulSoup(data.text)

In [ ]:
# Create a css selector
standings_table = soup.select('table.stats_table')[0]

In [27]:
# To get the a tags
links = standings_table.find_all('a')

In [28]:
# To get the href properity of each link
links = [l.get("href") for l in links]

In [29]:
# To filter for only squad links
links = [l for l in links if '/squads/' in l]

In [30]:
# To turn the links into full urls
team_urls = [f"https://fbref.com{l}" for l in links]

In [31]:
# To work with stats for a team
team_url = team_urls[0]

In [32]:
# To get the data for that team
data = requests.get(team_url)

In [33]:
# Turn scores and fixtures table into a dataframe
import pandas as pd

# Read the HTML content and search for the table
matches = pd.read_html(data.text, match='Scores & Fixtures')

In [34]:
# Initialise soup object
soup = BeautifulSoup(data.text)

In [35]:
# To access the a tags
links = soup.find_all('a')

In [36]:
# To get the href property of each link
links = [l.get("href") for l in links]

In [37]:
# To only get the links for the shooting stats
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [39]:
# To get the html of the shooting stats page
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
# To transform the html text into a pandas dataframe
shooting = pd.read_html(data.text, match="Shooting")[0]

In [44]:
# To drop the first level of index
shooting.columns = shooting.columns.droplevel()

In [ ]:
# To merge match data with shooting data
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
# To clean up the names of the teams
team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

'Liverpool'

In [65]:
# Setup a list of all the years we want to scrape
years = list(range(2024, 2020, -1))

In [66]:
# Initialise a list of data frames for a team in a season
all_matches = []

In [67]:
# Define the url we are going to start with
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [71]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(2)

IndexError: list index out of range

In [ ]:
# Combine all the dataframes
match_df = pd.concat(all_matches)

NameError: name 'all_matches' is not defined

In [ ]:
# Lowercase all the column names
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
# Write the data into a csv file
match_df.to_csv("matches.csv")